In [ ]:
Here’s a Python script that implements a Semantic Spotter using LangChain, tools, a retriever, a vector database (FAISS), Retrieval-Augmented Generation (RAG), and Gradio for the user interface. It uses an open-source language model (e.g., Groq) and demonstrates integration with all the mentioned components.

### Key Features of the Code

1. **Document Loading**: Loads and preprocesses documents from a specified folder.
2. **Vector Database**: Creates a FAISS index using `HuggingFaceEmbeddings`.
3. **LLM Initialization**: Uses `Groq` with an open-source model (e.g., `mixtral-8x7b-32768`).
4. **Retrieval-Augmented Generation**: Combines the LLM with the FAISS retriever in a `RetrievalQA` chain.
5. **Agent with Tools**: Initializes an agent to manage tool-based responses.
6. **Gradio Interface**: Provides an interactive web interface for users to interact with the chatbot.

### Instructions
- Replace `"path/to/your/documents"` with the path to your text documents.
- Ensure the required models (e.g., `sentence-transformers/all-MiniLM-L6-v2`, `EleutherAI/gpt-neo-1.3B`) are available or downloaded.
- Launch the code in an environment with GPU for better performance.

Let me know if you need help customizing or expanding the chatbot's capabilities!

In [14]:
pip install langchain llama-index  langchain_groq llama-index-llms-groq PyPDF2 gradio faiss-cpu faiss-gpu langchain-community sentence-transformers transformers langchain huggingface_hub groq

In [15]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.llms import HuggingFacePipeline
from langchain.agents import Tool, initialize_agent, AgentType
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from transformers import pipeline
from langchain.document_loaders import PyPDFLoader
from langchain.chains.question_answering import load_qa_chain
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from transformers import LlamaTokenizer, LlamaForCausalLM
from llama_index.llms.groq import Groq
from langchain_groq import ChatGroq
from groq import Groq
from google.colab import userdata
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
import torch
import subprocess
import json
import gradio as gr
import os

In [16]:
from PyPDF2 import PdfReader
from langchain.docstore.document import Document

In [17]:
import os

from google.colab import auth
auth.authenticate_user()

# Retrieve the secret
os.environ["GROQ_API_KEY"] = userdata.get("groq_api_key")

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
pdf_docs = "/content/drive/MyDrive/pdf/"
#txt_path = "/content/drive/MyDrive/cbse.txt"

In [20]:
def get_pdf_text(pdf_docs):
    text = ""
    for filename in os.listdir(pdf_docs):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(pdf_docs, filename)
            pdf_reader = PdfReader(pdf_path)
            for page in pdf_reader.pages:
                text += page.extract_text()
    return text

In [21]:
def get_text_chunks(text):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_text(text)
    return chunks

In [22]:
# Step 2: Create vector database
def create_vector_db(chunks):
    embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vector_db = FAISS.from_texts(chunks, embedding_model)
    return vector_db

In [23]:
# Step 3: Initialize the LLM
def initialize_llm():
    llm = ChatGroq(temperature=0, model_name="mixtral-8x7b-32768")

    return llm

In [24]:
# Step 4: Define retrieval-augmented generation (RAG) chain
def initialize_rag_chain(vector_db, llm):
    retriever = vector_db.as_retriever()
    qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, chain_type="stuff", return_source_documents=True )
    return qa_chain

In [25]:
# Step 5: Define agent with tools
def initialize_agent_with_tools(qa_chain):
    tool = Tool(
        name="Document Retriever",
        func=qa_chain.invoke,
        description="Retrieve relevant information from the document database."
    )
    tools = [tool]
    agent = initialize_agent(tools=tools, llm=llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, handle_parsing_errors=True, verbose=True)
    return agent

In [27]:
# Step 6: Gradio Interface
def chatbot_interface(agent):
    def chatbot_response(input_text):
        response = agent.run(input_text)
        return response

    gr.Interface(
        fn=chatbot_response,
        inputs=gr.Textbox(label="Enter your question:"),
        outputs=gr.Textbox(label="Response:"),
        title="Semantic Spotter",
        description="Ask questions about the loaded documents.",
    ).launch()

if __name__ == "__main__":

    print("Loading documents...")
    text = get_pdf_text(pdf_docs)

    print("Chunking documents...")
    chunks = get_text_chunks(text)

    print("Creating vector database...")
    vector_db = create_vector_db(chunks)

    print("Initializing language model...")
    llm = initialize_llm()

    print("Setting up RAG chain...")
    qa_chain = initialize_rag_chain(vector_db, llm)

    print("Initializing agent with tools...")
    agent = initialize_agent_with_tools(qa_chain)

    print("Launching chatbot...")
    chatbot_interface(agent)


Loading documents...
Chunking documents...
Creating vector database...
Initializing language model...
Setting up RAG chain...
Initializing agent with tools...
Launching chatbot...
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3cafebd232caa8d4e7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
